# 00 - Resumen Ejecutivo (One-Page)

Este notebook sintetiza el contexto y resultados clave del proyecto para lectura rapida.

## Disclaimer
> La base fue recabada en **2021** durante el auge de la pandemia COVID-19.
> La muestra es **pequena y no probabilistica** (178 respuestas).
> El objetivo actual es demostrar una implementacion profesional de Data Analyst con datos reales historicos.


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', 200)


In [ ]:
project_root = Path.cwd()
if not (project_root / 'data').exists() and (project_root.parent / 'data').exists():
    project_root = project_root.parent

processed_csv = project_root / 'data/processed/survey_analytics.csv'
if not processed_csv.exists():
    raise FileNotFoundError('No existe data/processed/survey_analytics.csv. Ejecuta primero src/data/clean_survey.py')

df = pd.read_csv(processed_csv)
print('Shape:', df.shape)
df.head(2)


In [ ]:
def pct(series, value):
    valid = series.dropna()
    if len(valid) == 0:
        return np.nan
    return round((valid == value).mean() * 100, 2)

kpis = {
    'Respuestas analizadas': len(df),
    '% impacto negativo': pct(df['q3_impact'], 'Negativa'),
    '% reporta problemas (q4=Si)': pct(df['q4_problems'], 'Si'),
    '% bienestar final No/Tal vez': round(df['q15_wellbeing_final'].isin(['No', 'Tal vez']).mean() * 100, 2),
}

kpi_df = pd.DataFrame(kpis.items(), columns=['Indicador', 'Valor'])
kpi_df


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

impact_order = df['q3_impact'].fillna('Sin dato').value_counts().index
sns.countplot(data=df, x='q3_impact', order=impact_order, ax=axes[0], palette='viridis')
axes[0].set_title('Percepcion del impacto de la pandemia en la muestra')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=20)

well_order = ['No', 'Tal vez', 'Si', 'Prefiero no responder.']
well_series = df['q15_wellbeing_final'].fillna('Sin dato')
well_order = [x for x in well_order if x in well_series.unique()] + [x for x in well_series.unique() if x not in well_order]
sns.countplot(x=well_series, order=well_order, ax=axes[1], palette='magma')
axes[1].set_title('Estado final de bienestar reportado')
axes[1].set_xlabel('')
axes[1].tick_params(axis='x', rotation=20)

plt.tight_layout()
plt.show()


In [ ]:
def chi_square(data, a, b):
    sub = data[[a, b]].dropna()
    table = pd.crosstab(sub[a], sub[b])
    chi2, p_value, dof, _ = chi2_contingency(table)
    return chi2, p_value, dof, len(sub)

tests = []
chi2, p, dof, n = chi_square(df, 'q3_impact', 'q15_wellbeing_final')
tests.append(['q3_impact vs q15_wellbeing_final', chi2, p, dof, n])
chi2, p, dof, n = chi_square(df, 'q4_problems', 'q15_wellbeing_final')
tests.append(['q4_problems vs q15_wellbeing_final', chi2, p, dof, n])

tmp = df.copy()
tmp['anxiety_high'] = np.where(tmp['q14_anxiety_score'] >= 3, 'Alta', 'No alta')
chi2, p, dof, n = chi_square(tmp, 'q4_problems', 'anxiety_high')
tests.append(['q4_problems vs anxiety_high', chi2, p, dof, n])

chi_df = pd.DataFrame(tests, columns=['Prueba', 'Chi2', 'p-value', 'dof', 'n'])
chi_df['Significativa (alpha=0.05)'] = chi_df['p-value'] < 0.05
chi_df


In [ ]:
from IPython.display import Markdown, display

k = dict(zip(kpi_df['Indicador'], kpi_df['Valor']))
sig = chi_df[chi_df['Significativa (alpha=0.05)'] == True]['Prueba'].tolist()

lines = []
lines.append('## Lectura Ejecutiva')
lines.append('')
lines.append(f"- La base incluye **{int(k['Respuestas analizadas'])} respuestas** (2021).")
lines.append(f"- El **impacto negativo** aparece en **{k['% impacto negativo']}%** de la muestra.")
lines.append(f"- El **reporte de problemas** alcanza **{k['% reporta problemas (q4=Si)']}%**.")
lines.append(f"- El segmento con **bienestar final en riesgo (No/Tal vez)** representa **{k['% bienestar final No/Tal vez']}%**.")
if sig:
    lines.append('- Asociaciones significativas detectadas:')
    for s in sig:
        lines.append(f"  - `{s}`")
lines.append('')
lines.append('**Conclusion:** caso util para demostrar metodologia profesional de Data Analyst sobre datos historicos reales.')

display(Markdown('\n'.join(lines)))
